In [ ]:
from google.cloud import bigquery
from google.oauth2 import service_account

import pandas as pd
import numpy as np
import math

from sklearn.model_selection import train_test_split

from scipy import stats
from dython.nominal import associations

import seaborn as sns
import matplotlib.pyplot  as plt

In [ ]:
train_name='train_incident'
validation_name='validation_incident'
test_name='test_incident'

In [ ]:
projectId='pongthorn'
client = bigquery.Client(project=projectId)

# projectId='smart-data-ml'
# credentials = service_account.Credentials.from_service_account_file(r'C:\Windows\smart-data-ml-91b6f6204773.json')
# client = bigquery.Client(credentials=credentials, project=projectId)

dataset_dw_id='SMartDW'
dataset_id='SMartML'

dw_table_id = f"{projectId}.{dataset_dw_id}.incident"

train_table_id=f"{projectId}.{dataset_id}.{train_name}"
val_tabel_id=f"{projectId}.{dataset_id}.{validation_name}"
test_tabel_id=f"{projectId}.{dataset_id}.{test_name}"

file_name="ML_Incident.csv"


In [ ]:
isLastUpdate=True

IsNewData=False

label='severity_id'
labelName='severity_name'

threshold_x_sd = 3  # 2.5/3/3.5


removeCols=['id','updated_at','imported_at']

dateCols=['open_datetime','close_datetime','response_datetime','resolved_datetime']

numbericCols=['count_detail','open_to_close_hour','open_to_response_hour','response_to_resolved_hour']
#numbericCols=['count_detail','open_to_close_hour']

cateCols=['sla','product_type','brand','service_type','incident_type']


# Explore and Transform Data

In [ ]:
def load_data_bq(sql:str):

 query_result=client.query(sql)
 df_all=query_result.to_dataframe()
 return df_all


In [ ]:
imported_to='2023-11-01'  # for test  date+1 of the day that you perform
#imported_to='2023-03-13' For production
#updated_to='2023-03-13' # For test

sql_all=f"""
SELECT  id,severity_id,severity_name
,count_detail
,sla,product_type,brand,service_type,incident_type
,open_datetime,  close_datetime, response_datetime,resolved_datetime
,updated_at,imported_at

FROM `{dw_table_id}`    

WHERE imported_at< '{imported_to}'

order  by imported_at
"""
#WHERE updated_at< '{updated_to}'
#WHERE imported_at< '{imported_to}'

# copy sql statement to check whether to be valid or not
print(sql_all)

In [ ]:
print("Load data from Bigquery")
df_all=load_data_bq(sql_all)
if len(df_all)==0:
 print("No records from bigquery")  
 quit()
else:
 print(df_all.shape)   

In [ ]:
df_all=df_all.drop_duplicates(subset=['id'],keep='last')

no_original_rows=len(df_all)

df_all.info()

# Manage Numberic Cols
Converting the data of these columns to hourly time period., it is more informative


In [ ]:
start_end_list=[ ['open_datetime','close_datetime'],['open_datetime','response_datetime'],['response_datetime','resolved_datetime']]
listDiffDateDeltaCols=[]
listDiffHourCols=[]
for item  in  start_end_list:
   diff_str=f"{item[0]}_to_{item[1]}" 
   diff_str=diff_str.replace('_datetime','')  
   listDiffDateDeltaCols.append(diff_str)
   df_all[diff_str]=df_all[item[1]]-df_all[item[0]]
    
   diff_hour=f'{diff_str}_hour'
   listDiffHourCols.append(diff_hour)
   df_all[diff_hour] = df_all[diff_str].apply(lambda x:  x.total_seconds() / (60*60) if x is not np.nan else np.nan  )


#df_all[listDiffHourCols].describe()
df_all[listDiffHourCols].describe(percentiles=[.95,.75,.50,.25,.05])

# Remove outlier on Time Interval to service

In [ ]:
# https://www.geeksforgeeks.org/z-score-for-outlier-detection-python/
xScoreDiffHourCols=[]
for col in listDiffHourCols:
  z_col=f"zscore_{col}"   
  df_all[z_col] = np.abs(stats.zscore(df_all[col]))   
  xScoreDiffHourCols.append(z_col)  
#remove oulier

for col in  xScoreDiffHourCols:
  df_all=df_all.query(f"{col}<@threshold_x_sd")


no_rows_after_removing_outlier=len(df_all)
pct_row_decrease=round( (no_original_rows-no_rows_after_removing_outlier)/no_original_rows*100 ,0)

print(f"% remove data {pct_row_decrease}")

print(df_all.info())

# Find any rows  contain zero time period
## it doesn't make any sense on real world 

In [ ]:
# if>5%  remove column, else judge to remove 0 value 
# remove coundetail because mostly, there are few value 1,2 items
allRows=df_all.shape[0]
for col in numbericCols:
    zeroRows=len(df_all.query(f"{col}==0"))
    pctZeroHour=round(zeroRows/allRows*100,1)
    print(f"No. 0-hour row on {col} = {zeroRows}({pctZeroHour} %)")
    
  # open_to_response_hour =o is not realistic  ==> move columns  
# No. 0-hour row on open_to_response_hour = 456(19.2 %) , it is relevant to site manage's behaviod
# They get incident call from customer  and provide engineer to response completely
# afte that they will enter the application to record case 

# Bining Range

function is used to separate the array elements into many different ranges . 
The cut function is mainly used to perform statistical analysis on scalar data. 

In [ ]:
# https://www.geeksforgeeks.org/pandas-cut-method-in-python/


def explore_ranges_numberic_val(col,rangeList):
    print(col)
    rangeList.sort()
    return pd.cut(df_all[col],rangeList, right=True).value_counts()
range1= [0,24, 168, 360, 720,math.floor(df_all['open_to_close_hour'].max())]
print(explore_ranges_numberic_val('open_to_close_hour',range1))
print("=======================================================")

range2= [0,24, 168, 360, 720,math.floor(df_all['response_to_resolved_hour'].max())]
print(explore_ranges_numberic_val('response_to_resolved_hour',range2))
print("=======================================================")

# range3= [0,4, 8, 12, 24,math.floor(df_all['open_to_response_hour'].max())]
# print(explore_ranges_numberic_val('open_to_response_hour',range3))
# print("=======================================================")

range4= [0,2, 4,math.floor(df_all['count_detail'].max())]
print(explore_ranges_numberic_val('count_detail',range4))
print("=======================================================")

# we can convert hour to range   
# (0, 24] =by 1 day =best
# (24, 168] =  1day -1 week  =good
# (168, 360]=  1week- 15 days(half a month) =fair
# (360, 720]= 15 dasy-1 month =bad
# (720, 2349]=1 month-1 Q =worst

# open_to_close_hour ,response_to_resolved_hour , mostly  we can complate by 1 day (0, 24]
# there are few cased that take long to close incident (360, 720]   15 day to  1month

In [ ]:
#Anyway , We will have 2 columns left 
print("remove some cate featurs that have undeisred data")

# open_to_response_hour :No. 0-hour row on open_to_response_hour = 456(19.2 %)
# ount_detail  : 80%  (0, 2] =1790 You'll notice that most of the data is clustered in (0, 2] detail  only
numColsRemove=['open_to_response_hour','count_detail'] 
numbericCols=[x for x in numbericCols if x not in numColsRemove]   
numbericCols

In [ ]:
# remove response_to_resolved_hour ,it doesn't make any sense
# No. 0-hour row on response_to_resolved_hour = 99(4.2 %)
df_all=df_all.query('response_to_resolved_hour!=0')
df_all.info()

# Drop unnessay columns

In [ ]:
removeCols=removeCols+dateCols+xScoreDiffHourCols+listDiffDateDeltaCols+numColsRemove
df_all=df_all.drop(columns=removeCols)
df_all.info()

In [ ]:
df_all[numbericCols].describe(percentiles=[.95,.75,.50,.25,.05])
# 95% vs max

# Plot Numeric  columns

In [ ]:
df_all[numbericCols].plot(kind='box',subplots=True,layout=(1,len(numbericCols)),sharex=False, sharey=False,figsize=(10,5))
plt.show()

In [ ]:
pd.plotting.scatter_matrix(df_all[numbericCols], alpha=0.2,figsize=(10,6))

In [ ]:
corr_df = df_all[numbericCols].corr ()  
corr_df = corr_df .round(2)

fig = plt.figure(figsize = (10, 6))
heatmap = sns. heatmap(corr_df, annot = True) 
plt. show ()

# Plot Categorical Cols

In [ ]:
VBar=['brand','incident_type']
HBar=[ x for x in cateCols if x not in VBar]
HBar.insert(0,'severity_name')

In [ ]:
for col in HBar:
    fig , ax = plt.subplots(figsize=(15,5))
    ax =sns.countplot(x=col, data=df_all,)
    for p in ax.patches:
       ax.annotate('{:.1f}'.format(p.get_height()), (p.get_x()+0.25, p.get_height()+0.01))
    plt.title(col.title())
    plt.show()

In [ ]:
for col in VBar:
    fig , ax = plt.subplots(figsize=(10,20))
    ax =sns.countplot(y=col, data=df_all)
    plt.title(col.title())
    plt.show()

In [ ]:
# all of them are not hardly relevant, it is hight variance
complete_correlation= associations(df_all[ cateCols ], figsize=(15,8))

In [ ]:
print(df_all.info())
df_all.to_csv(f"data/{file_name}",index=False)
#bucket_name="gs://smart-ml-pongthorn"
# df_all.to_csv(f"{bucket_name}/{file_name}",index=False) 

# Build Training DataSet

In [ ]:
# create label
map_severity_to_class={'Cosmetic':0,'Minor': 1, "Major": 2, "Critical": 3}
print(f"Map severity Name to LabelCode: {str(map_severity_to_class)}")

In [ ]:
df_all['label_multi_severity'] =df_all['severity_name'].map(map_severity_to_class) 
def map_4to2_serverity(severity_id):
    if severity_id==1 or severity_id==2:
       return 1
    else:
       return 0 
df_all['label_binary_severity'] =df_all['severity_id'].apply(map_4to2_serverity)

print( list(df_all['label_multi_severity'].unique()))
print(df_all['label_binary_severity'].unique())
df_all.head()

In [ ]:
#Split data into 3 dataset to BQ
len_all=len(df_all)
print(f"All Data = {len_all}")

# Split the 80% of total as train
# The remaining  will be splited equally 50% for valuation and the rest of later part is test

train, val = train_test_split(df_all,test_size=0.2,random_state=1000,shuffle=True)
val,test = train_test_split(val, test_size=0.5,random_state=1000,shuffle=True)

len_all=len(df_all)
len_train=len(train)
len_val=len(val)
len_test=len(test)
print(f'{len_train} =train examples ({round(len_train/len_all*100,1)}%)')
print(f'{len_val} =val examples ({round(len_val/len_all*100,1)}%)')
print(f'{len_test} =test examples ({round(len_test/len_all*100,1)}%)')

In [ ]:
print(f"pct removed data ={(no_original_rows-len(df_all))/no_original_rows*100}")
#it is pretty close to 5%   , at most 10% is removable

In [ ]:
def loadDataFrameToBQ(table_id,dfx):
    try:
        job_config = bigquery.LoadJobConfig(
            write_disposition="WRITE_TRUNCATE",
        )

        job = client.load_table_from_dataframe(
            dfx, table_id, job_config=job_config
        )
        job.result()  # Wait for the job to complete.
        print(f"{len(dfx)} rows imported to {table_id} successfully")

    except BadRequest as e:
        print("Bigquery Error\n")
        for e in job.errors:
            print('ERROR: {}'.format(e['message']))
            
# save to csv file to load faster than bigquery
train.to_csv(f"data/{train_name}.csv",index=False)
val.to_csv(f"data/{validation_name}.csv",index=False)
test.to_csv(f"data/{test_name}.csv",index=False)

#write_disposition="WRITE_TRUNCATE"
#google will create table and load data into them automaticall 
loadDataFrameToBQ(f"{train_table_id}",train)
loadDataFrameToBQ(f"{val_tabel_id}",val)
loadDataFrameToBQ(f"{test_tabel_id}",test)   
    